In [1]:
with open('inputs/11.txt') as file:
    input = file.read()


In [2]:
TRACE = 0
def run(mem):
    i = 0
    rel = 0
    def get(j,m):
        p = i+j
        if m == 0:
            p = mem[p]
        elif m == 2:
            p = mem[p] + rel
        return p
    while 1:
        s = ('0000'+str(mem[i]))[-5:]
        a,b,c,op = int(s[0]), int(s[1]), int(s[2]), int(s[3:])
        if op == 1:
            x = get(1,c)
            y = get(2,b)
            p = get(3,a)
            if TRACE: print(f'{i}: {mem[x]}+{mem[y]}={mem[x] + mem[y]} @ {p}')
            mem[p] = mem[x] + mem[y]
            i += 4
        elif op == 2:
            x = get(1,c)
            y = get(2,b)
            p = get(3,a)
            if TRACE: print(f'{i}: {mem[x]}*{mem[y]}={mem[x] * mem[y]} @ {p}')
            mem[p] = mem[x] * mem[y]
            i += 4
        elif op == 3:
            p = get(1,c)
            mem[p] = yield 'input'
            if TRACE: print(f'{i}: input {mem[p]} @ {p}')
            i += 2
        elif op == 4:
            p = get(1,c)
            if TRACE: print(f'{i}: out {mem[p]} @ {p}')
            yield mem[p]
            i += 2
        elif op == 5:
            x = get(1,c)
            y = get(2,b)
            if TRACE: print(f'{i}: if {mem[x]} jump {mem[y]}')
            if mem[x]:
                i = mem[y]
            else:
                i += 3
        elif op == 6:
            x = get(1,c)
            y = get(2,b)
            if TRACE: print(f'{i}: if not {mem[x]} jump {mem[y]}')
            if not mem[x]:
                i = mem[y]
            else:
                i += 3
        elif op == 7:
            x = get(1,c)
            y = get(2,b)
            p = get(3,a)
            if TRACE: print(f'{i}: {mem[x]} < {mem[y]} => {p}')
            mem[p] = int(mem[x] < mem[y])
            i += 4
        elif op == 8:
            x = get(1,c)
            y = get(2,b)
            p = get(3,a)
            if TRACE: print(f'{i}: {mem[x]} == {mem[y]} => {p}')
            mem[p] = int(mem[x] == mem[y])
            i += 4
        elif op == 9:
            x = get(1,c)
            if TRACE: print(f'{i}: rel = {rel} + {mem[x]}')
            rel += mem[x]
            i += 2
        elif op == 99:
            return
        else:
            raise Exception('Unknown opcode', op)
            break


In [3]:
def run2(init=0):
    prog = list(map(int, input.split(','))) + [0]*10000

    p = 0
    d = 0
    steps = [-1j,1,1j,-1]
    rect = [0,0]
    paint = {p:init}
    proc = run(prog)
    try:
        while 1:
            next(proc)
            paint[p] = proc.send(paint[p] if p in paint else 0)
            rot = next(proc)
            d = (d+1)%4 if rot else (d+3)%4
            p += steps[d]
            rect[0] = complex(min(p.real,rect[0].real),min(p.imag,rect[0].imag))
            rect[1] = complex(max(p.real,rect[1].real),max(p.imag,rect[1].imag))
    except StopIteration:
        pass
    return paint,rect

paint,rect = run2()
len(paint)


2041

In [4]:
def draw(paint, rect):
    img = []
    for y in range(int(rect[0].imag),int(rect[1].imag+1)):
        row = ''
        for x in range(int(rect[0].real),int(rect[1].real+1)):
            p = complex(x,y)
            row += '⬜' if p in paint and paint[p] else '⬛'
        img.append(row)
    print('\n'.join(img))

draw(paint,rect)

⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬜⬜⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛
⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬜⬜⬛⬛⬛⬛⬛⬛⬜⬜⬜⬛⬜⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛
⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬜⬛⬛⬛⬜⬛⬛⬛⬜⬜⬜⬛⬛⬛⬜⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛
⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬜⬛⬛⬜⬛⬛⬛⬜⬛⬛⬜⬛⬜⬜⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛
⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬜⬜⬜⬜⬜⬜⬜⬜⬜⬛⬜⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛
⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬜⬛⬛⬛⬛⬛⬛⬛⬛⬛⬜⬛⬜⬜⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛
⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬜⬛⬜⬜⬛⬛⬜⬜⬜⬜⬛⬛⬜⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛
⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬜⬛⬜⬛⬜⬛⬛⬛⬛⬛⬜⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛
⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬜⬛⬛⬛⬛⬛⬛⬛⬛⬜⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬜⬛⬜⬛⬛⬛⬛⬛⬛⬛
⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬜⬜⬜⬛⬜⬜⬛⬛⬜⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬜⬜⬜⬜⬛⬛⬛⬛⬛⬛⬛
⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬜⬛⬜⬛⬛⬛⬜⬛⬜⬜⬛⬜⬛⬜⬛⬛⬜⬛⬛⬛⬛⬛⬜⬜⬛⬛⬛⬛⬛⬛⬛
⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬜⬛⬛⬛⬜⬜⬛⬜⬛⬛⬜⬜⬛⬜⬛⬜⬛⬛⬛⬛⬜⬜⬛⬛⬜⬛⬜⬛⬛⬛⬛⬛
⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬜⬜⬛⬜⬛⬜⬜⬜⬜⬛⬜⬜⬛⬜⬜⬛⬛⬛⬜⬛⬛⬛⬛⬛⬛⬜⬜⬛⬛⬛⬛
⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬜⬜⬜⬜⬛⬛⬛⬜⬜⬜⬛⬛⬛⬜⬛⬜⬛⬜⬜⬛⬛⬛⬜⬛⬛⬜⬜⬛⬛⬛⬛⬛
⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛⬛

In [5]:
paint,rect = run2(1)
draw(paint,rect)

⬛⬜⬜⬜⬜⬛⬜⬜⬜⬛⬛⬜⬜⬜⬜⬛⬜⬜⬜⬛⬛⬜⬛⬛⬜⬛⬜⬜⬜⬜⬛⬜⬜⬜⬜⬛⬜⬜⬜⬛⬛⬛⬛
⬛⬛⬛⬛⬜⬛⬜⬛⬛⬜⬛⬛⬛⬛⬜⬛⬜⬛⬛⬜⬛⬜⬛⬜⬛⬛⬜⬛⬛⬛⬛⬛⬛⬛⬜⬛⬜⬛⬛⬜⬛⬛⬛
⬛⬛⬛⬜⬛⬛⬜⬛⬛⬜⬛⬛⬛⬜⬛⬛⬜⬛⬛⬜⬛⬜⬜⬛⬛⬛⬜⬜⬜⬛⬛⬛⬛⬜⬛⬛⬜⬛⬛⬜⬛⬛⬛
⬛⬛⬜⬛⬛⬛⬜⬜⬜⬛⬛⬛⬜⬛⬛⬛⬜⬜⬜⬛⬛⬜⬛⬜⬛⬛⬜⬛⬛⬛⬛⬛⬜⬛⬛⬛⬜⬜⬜⬛⬛⬛⬛
⬛⬜⬛⬛⬛⬛⬜⬛⬜⬛⬛⬜⬛⬛⬛⬛⬜⬛⬛⬛⬛⬜⬛⬜⬛⬛⬜⬛⬛⬛⬛⬜⬛⬛⬛⬛⬜⬛⬜⬛⬛⬛⬛
⬛⬜⬜⬜⬜⬛⬜⬛⬛⬜⬛⬜⬜⬜⬜⬛⬜⬛⬛⬛⬛⬜⬛⬛⬜⬛⬜⬜⬜⬜⬛⬜⬜⬜⬜⬛⬜⬛⬛⬜⬛⬛⬛
